# OCR con Tesseract

Tesseract convierte imágenes en texto en cuatro pasos básicos:

- Preprocesado y binarización: Convierte la imagen a blanco y negro con umbrales adaptativos para destacar los trazos de tinta y reducir ruido.

- Segmentación: Encuentra los contornos (“blobs”) de los caracteres, agrupa blobs en líneas y luego en palabras usando análisis de componentes conectados y proyección vertical/horizontal.

- Reconocimiento: Hasta la versión 3 usaba un clasificador basado en patrones y árboles de decisiones. Desde la versión 4 (la que usamos en este análisis) emplea redes LSTM bidireccionales que procesan la secuencia de píxeles línea por línea y generan la probabilidad de cada carácter.

- Post-procesado: Aplica un modelo de lenguaje y diccionarios para corregir errores típicos (p. ej. confundir “0” y “O”), calcular la confianza y producir el texto final.

La documentación de la librería está disponible en https://tesseract-ocr.github.io/tessdoc/tess4/NeuralNetsInTesseract4.00.html?utm_source=chatgpt.com 

## Lectura y extracción de texto
Este script extrae texto de PDFs escaneados usando **PyMuPDF** y utilizando Optical Caracter Recognition de **Tesseract**.  
Antes de correrlo, recomendamos leer los requisitos en readme_ocr-pdf-extractor de este repositorio.

1) Vamos a instalar las librerías necesarias para el proyecto

In [ ]:
#Requisitos previos (solo una vez):
#%pip install pymupdf pillow pytesseract
#   Instalar Tesseract y el idioma español (spa.traineddata) en la computadora (ver instrucciones en el readme)

In [1]:
import os
import io
import fitz  # PyMuPDF
from PIL import Image, ImageDraw
import pytesseract
from pathlib import Path
from pydantic import BaseModel
from typing import Optional

In [ ]:
import csv, json, re, unicodedata
from pathlib import Path
import requests
from requests.exceptions import ReadTimeout
from pydantic import ValidationError
from tqdm import tqdm

2) Definimos una ruta a los archivos. 

In [3]:
PDF_FOLDER    = Path("cds_ejemplo")
OUTPUT_FOLDER = Path("outputs") / "Extraccion_Tesseract"

OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)


3) Vamos a definir un diccionario con los parámetros del OCR:

- DPI: resolución para renderizar la página (default 300).
- Lang: idioma OCR (español).
- Mode: Dejamos la configuración default que posee un perfil OCR con LSTM.

In [4]:
LANG = "spa"     
DPI = 300      
PSM = 6          

4) Definimos una la función *pdf_to_images* que recorre el pdf y convierte cada página en imagen, extrae el texto  con PyMuPDF y lo guarda. Este paso es importante para después poder aplicar el OCR.  
Luego, definimos la función *ocr_pdf* para procesar los PDFs y guardar el TXT. 

In [5]:
def pdf_to_images(pdf_path: Path, dpi: int):
    doc = fitz.open(pdf_path)
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)
    for page in doc:
        pix = page.get_pixmap(matrix=mat, alpha=False)
        yield Image.frombytes("RGB", [pix.width, pix.height], pix.samples)


def ocr_pdf(pdf_path: Path) -> str:
    texto = []
    for img in pdf_to_images(pdf_path, DPI):
        texto.append(
            pytesseract.image_to_string(img, lang=LANG, config=f"--psm {PSM}")
        )
    return "\n".join(texto)


5) Ejecutamos el OCR en todos los PDFs de la carpeta y guardamos el texto extraído en archivos .txt

In [6]:
def main():
    in_dir = Path(PDF_FOLDER)
    out_dir = Path(OUTPUT_FOLDER)
    out_dir.mkdir(parents=True, exist_ok=True)

    pdfs = list(in_dir.glob("*.pdf"))
    if not pdfs:
        print(f"No se encontraron PDFs en {in_dir.resolve()}")
        return

    for pdf in pdfs:
        print(f"Procesando {pdf.name}…")
        texto = ocr_pdf(pdf)
        (out_dir / f"{pdf.stem}.txt").write_text(texto, encoding="utf-8")
    print("Listo. Archivos guardados en", out_dir.resolve())


if __name__ == "__main__":
    main()


Procesando Belen_payway.pdf…
Procesando Bruno23689270.pdf…
Procesando Bruno400212294002.pdf…
Procesando Bruno400212294002_1.pdf…
Procesando Coyle401547680601.pdf…
Procesando EVERTEC30707869484.pdf…
Procesando Galicia_Giuseppe.pdf…
Procesando hsbc_Aquino.pdf…
Procesando hsbc_Aquino2.pdf…
Procesando hsbc_Aquino3.pdf…
Procesando Mallo12980371.pdf…
Procesando Pantano28462989.pdf…
Procesando Santander_evertec.pdf…
Listo. Archivos guardados en D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\Trabajo_Final\outputs\Extraccion_Tesseract


## Extracción de variables con Llama 3
Vamos a utilizar structured outputs para extraer información de los txt y transformarlos en variables. Esto nos facilitará la evaluación de la calidad del OCR.

In [12]:
TXT_FOLDER = Path("outputs") / "Extraccion_Tesseract"
OUTPUT_CSV = Path("entidades_extraidas_Tesseract.csv")
PROMPT_FILE = "prompt_2.txt"
MODEL       = "llama3:latest"
OLLAMA_URL  = "http://localhost:11434/api/generate"
TEMPERATURE = 0.0
TIMEOUT     = 900
MAX_CHARS   = 9000

In [13]:
class Entity(BaseModel):
    Remitente: Optional[str] = None
    DNI: Optional[str] = None
    CUIT_CUIL: Optional[str] = None
    Cuerpo: str


Vamos a generar una función para eliminar caracteres especiales y saltos de línea del texto extraído

In [14]:
CLEAN_RE = re.compile(r"[^\w\s.,;:()@€$%/-]")

def clean_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = CLEAN_RE.sub("", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

Definimos una función para extraer entidades con llama

In [15]:
JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "Remitente": {"type": ["string", "null"]},
        "DNI": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "CUIT_CUIL": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "Cuerpo": {"type": "string"},
    },
    "required": ["Cuerpo"],
    "additionalProperties": False,
}

with open(PROMPT_FILE, encoding="utf-8") as f:
    PROMPT = f.read()

def call_ollama(text: str) -> dict:
    payload = {
        "model": MODEL,
        "prompt": f"<|system|>\n{PROMPT}<|end|>\n<|user|>\n{text[:MAX_CHARS]}<|end|>\n<|assistant|>",
        "format": "json",
        "stream": False,
        "options": {"temperature": TEMPERATURE, "json_schema": JSON_SCHEMA},
    }
    try:
        r = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    except ReadTimeout:
        raise TimeoutError("Timeout de Ollama")
    r.raise_for_status()
    return json.loads(r.json()["response"])

def main():
    dir_path = Path(TXT_FOLDER)
    if not dir_path.is_dir():
        raise SystemExit(f"Ruta no encontrada: {dir_path}")

    rows = []
    for file in tqdm(sorted(dir_path.glob("*.txt")), desc="TXT"):
        raw = file.read_text(encoding="utf-8", errors="ignore")
        cleaned = clean_text(raw)
        try:
            data = call_ollama(cleaned)
            ent = Entity.model_validate(data)
            rows.append({
                "ARCHIVO": file.name,
                "Remitente": (ent.Remitente or "NaN").strip() or "NaN",
                "DNI": re.sub(r"\D", "", ent.DNI or "") or "NaN",
                "CUIT_CUIL": re.sub(r"\D", "", ent.CUIT_CUIL or "") or "NaN",
                "Cuerpo": ent.Cuerpo.strip() or "NaN",
            })
        except Exception as e:
            print(f"❌ {file.name}: {e}")
    if rows:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=rows[0].keys()).writeheader(); csv.DictWriter(f, fieldnames=rows[0].keys()).writerows(rows)
        print(f"✅ CSV generado en {OUTPUT_CSV} ({len(rows)} filas)")
    else:
        print("No se extrajeron entidades válidas.")

Ejecutamos

In [16]:
if __name__ == "__main__":
    main()

TXT: 100%|██████████| 13/13 [1:29:20<00:00, 412.33s/it]


✅ CSV generado en entidades_extraidas_Tesseract.csv (13 filas)


La extracción de entidades logró detectar bien el nombre del remitente y el cuerpo, pero presenta dificultades para leer DNI y CUIT. Esto podría resolverse utilizando un modelo más grande, que reconozca con más facilidad ese tipo de variables. 